In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

### 1. Data loading and preprocessing

In [2]:
# Scaling the image (dividing a value of each pixels by 255, in order to get values from 0 to 1)  
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

In [3]:
# Loading the dataset
mnist_dataset, mnist_info = tfds.load(name='mnist', shuffle_files=True, with_info=True, as_supervised=True)

# Dividing the dataset into train and test sets
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

# Choosing the size for validation set (as 10% of the train set's size)
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

# Saving the size of test set into variable
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

# Scaling train and validation data (dividing by 255)
scaled_train_and_validation_data = mnist_train.map(scale)

# Scaling test data (dividing by 255)
scaled_test_data = mnist_test.map(scale)

# Shuffling the data
buffer_size = 100

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(buffer_size)

# Dividing into train and validation datasets
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

# Batching the train data
mini_batch_size = 100
train_data = train_data.batch(mini_batch_size)

# Batching validation and test data: they don't need to be batched, 
# but the model will require all data in batch form, so we take batch size as the whole set's size   
validation_data = validation_data.batch(num_validation_samples)
test_data = scaled_test_data.batch(num_test_samples)

# make the same shape and object properties for validation data as for the train data (itarable)
validation_inputs, validation_targets = next(iter(validation_data))

### 2. Model

In [4]:
input_size = 784
output_size = 10
hidden_layer_size = 400

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),                            
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ])


# Defining optimizer and loss function.
# sparse_categorical_crossentropy includes making a one-hot encoding for the data.
# Creating en optimizer customly in order to tune the learning rate

custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=custom_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### 3. Training the model

In [5]:
num_epochs = 6

model.fit(train_data, epochs=num_epochs, validation_data=(validation_inputs, validation_targets), validation_steps=1, verbose=2)

Epoch 1/6
540/540 - 10s - loss: 0.4490 - accuracy: 0.8811 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/6
540/540 - 10s - loss: 0.1515 - accuracy: 0.9552 - val_loss: 0.1585 - val_accuracy: 0.9530
Epoch 3/6
540/540 - 10s - loss: 0.1034 - accuracy: 0.9691 - val_loss: 0.1276 - val_accuracy: 0.9622
Epoch 4/6
540/540 - 10s - loss: 0.0756 - accuracy: 0.9785 - val_loss: 0.1104 - val_accuracy: 0.9658
Epoch 5/6
540/540 - 10s - loss: 0.0569 - accuracy: 0.9842 - val_loss: 0.1054 - val_accuracy: 0.9668
Epoch 6/6
540/540 - 10s - loss: 0.0434 - accuracy: 0.9881 - val_loss: 0.1000 - val_accuracy: 0.9703


### 4. Testing the model

In [6]:
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

1/1 [==============================] 0.0910 - accuracy: 0.97 - 2s 2s/step - loss: 0.0910 - accuracy: 0.9729
Test loss: 0.09. Test accuracy: 97.29%
